In [1]:
#!pip install pyautogui
#!pip install selenium
#import pyautogui

In [3]:
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
from tqdm import tqdm
import time

# DATA CASE 1
country     = "Marrakech"
filename    = "data_"+country
link        = "https://www.google.com/maps/search/Banque+marakech/@31.6278886,-8.0211712,13z/data=!3m1!4b1?entry=ttu"
banks_name  = []
banks_info  = []
full_review = []
verbose     = False

# UTILS
class Utils:
    @staticmethod
    def is_phone_number(text):
        phone_pattern = re.compile(r"^\+?\d[\d\s-]{8,}\d$")
        return bool(phone_pattern.match(text))
    
    @staticmethod
    def is_website_url(text):
        url_pattern = re.compile(
            r'^(https?:\/\/)?'
            r'([\da-z\.-]+)\.' 
            r'([a-z\.]{2,6})' 
            r'([\/\w \.-]*)*\/?$'
        )
        return bool(url_pattern.match(text))

def primary_search(browser):
    action = ActionChains(browser)
    a = browser.find_elements(By.CLASS_NAME, "hfpxzc")
    
    last_len = len(a)
    same_len_count = 0
    
    while True:
        # Scroll down to the last element
        #try catch to avoid StaleElementReferenceException and continue the loop
        +
        
        scroll_origin = ScrollOrigin.from_element(a[-1])
        action.scroll_from_origin(scroll_origin, 0, 1000).perform()
        time.sleep(2)  # Wait for new results to load
        
        a = browser.find_elements(By.CLASS_NAME, "hfpxzc")
        
        if len(a) == last_len:
            same_len_count += 1
            if same_len_count > 5:
                break
        else:
            last_len = len(a)
            same_len_count = 0
    
    return a, action

def throw_error(e):
    print(f"Error: {e}")

def extract_review(browser, action):
    # Click on "review button"
    try:
        tab_action = browser.find_elements(By.CLASS_NAME, "hh2c6")
        advice_btn = tab_action[1]
        action.move_to_element(advice_btn).click().perform()
        time.sleep(2)
    except Exception as e:
        throw_error(e)

    # Scroll down until no more data is loading while loading reviews
    reviews_blocs = browser.find_elements(By.CLASS_NAME, "jJc9Ad") 
    last_reviews_count = len(reviews_blocs)
    if verbose:
        print(last_reviews_count)
    _same = 0
    while True:
        scroll_origin = ScrollOrigin.from_element(reviews_blocs[-1])
        action.scroll_from_origin(scroll_origin, 0, 1000).perform()
        time.sleep(2)
        reviews_blocs = browser.find_elements(By.CLASS_NAME, "jJc9Ad") 

        if len(reviews_blocs) == last_reviews_count:
            _same += 1
            if _same > 3:
                break
        else:
            last_reviews_count = len(reviews_blocs)
            _same = 0
        
    # Extract the reviews
    reviews = []
    for bloc in reviews_blocs:
        html_content = bloc.get_attribute('outerHTML')
        html_content = BeautifulSoup(html_content, 'html.parser')
        try:
            soup = html_content.find('div', {"class": "d4r55"})
            reviewer_name = soup.text
            
            soup = html_content.findAll('span', {"class": "hCCjke google-symbols NhBTye elGi1d"})
            reviewer_star = len(soup)

            soup = html_content.find('span', {"class": "wiI7pd"})
            reviewer_text = soup.text if soup else "NAN"

            soup = html_content.find('span', {"class": "rsqaWe"})
            reviewer_publish_data = soup.text

            soup = html_content.find('span', {"class": "pkWtMe"})
            reviewer_like_reaction = soup.text if soup else 0
            
            soup = html_content.find('button', {"class": "WEBjve"})
            reviewer_profil_link = soup.attrs.get('data-href')

            soup = html_content.findAll('div', {"class": "wiI7pd"})
            if soup != None or len(soup)!=0:
                chat = [msg.text for msg in soup]
                reviewer_owner_reply = "**".join(chat)
            else:
                reviewer_owner_reply = "NAN"

            soup = html_content.find('span', {"class": "DZSIDd"})
            reviewer_owner_reply_date = soup.text if soup else "NAN"

            reviews.append((reviewer_name, reviewer_star, reviewer_text, reviewer_publish_data, reviewer_like_reaction, reviewer_profil_link, reviewer_owner_reply, reviewer_owner_reply_date))
        except Exception as e:
            throw_error(e)
            continue

    return reviews

def extract(browser, sites, action):
    for i in tqdm(range(len(sites))):
        try:
            scroll_origin = ScrollOrigin.from_element(sites[i])
            action.scroll_from_origin(scroll_origin, 0, 100).perform()
            action.move_to_element(sites[i]).perform()
            sites[i].click()
            time.sleep(2)
        except StaleElementReferenceException:
            print("StaleElementReferenceException occurred. Retrying...")
            sites = primary_search(browser) 
            continue
        source = browser.page_source
        soup = BeautifulSoup(source, 'html.parser')
        try:
            Name_Html = soup.findAll('h1', {"class": "DUwDvf lfPIob"})
            name = Name_Html[0].text
            if name not in banks_name:
                # Scrape Bank information
                banks_name.append(name)
                infos = soup.findAll('div', {"class": "Io6YTe"})
                phone = "Not available"
                for info in infos:
                    if Utils.is_phone_number(info.text):
                        phone = info.text
                address = infos[0].text if infos else "Not available"
                website = "Not available"
                for info in infos:
                    if Utils.is_website_url(info.text):
                        website = info.text
                if verbose:
                    print([name, phone, address, website])
                bank_details = (name, phone, address, website)
                
                # Scrape reviews
                reviews = extract_review(browser, action)
                for i in range(len(reviews)):
                    full_review.append(bank_details + reviews[i])

                # Save record
                df = pd.DataFrame(full_review, columns=['Bank_Name', 'Bank_Phone_number', 'Bank_Address', 'Bank_Website', 'Reviewer_Nane', 'Reviewer_Sart', 'Reviewer_Text', 'Reviewer_Publish_Date', 'Reviewer_Like_Reaction', 'Reviewer_Profil_Link', 'Reviewer_Owner_Reply', 'Reviewer_Owner_Reply_Date'])
                df.to_csv(filename + '.csv', index=False, encoding='utf-8')
        except Exception as e:
            # Alert maintanier
            throw_error(e)
            continue

# ROBOT ON ACTION
chrome_options = Options()
chrome_options.add_argument("--lang=ar")
#chrome_options.add_argument("--headless")
browser = webdriver.Chrome(options=chrome_options)
browser.get(link)
time.sleep(10)


sites, action = primary_search(browser)
extract(browser, sites, action)

browser.quit()

 70%|███████   | 14/20 [41:24<46:38, 466.37s/it]

Error: list index out of range


 90%|█████████ | 18/20 [1:21:22<09:02, 271.25s/it]


ElementClickInterceptedException: Message: element click intercepted: Element <a class="hfpxzc" aria-label="Guichet automatique La Bank Populaire" href="https://www.google.com/maps/place/Guichet+automatique+La+Bank+Populaire/data=!4m7!3m6!1s0xdafee661d79e1a9:0x731d6f7590cf5766!8m2!3d31.6335857!4d-7.992028!16s%2Fg%2F11c53gmp66!19sChIJqeF5HWburw0RZlfPkHVvHXM?authuser=0&amp;hl=fr&amp;rclk=1" jsaction="pane.wfvdle89;focus:pane.wfvdle89;blur:pane.wfvdle89;auxclick:pane.wfvdle89;keydown:pane.wfvdle89;clickmod:pane.wfvdle89" jslog="12690;track:click,contextmenu;mutable:true;metadata:WyIwYWhVS0V3alZ1OURKemFhR0F4VkpZUEVESGQwVUR1b1E4QmNJamdVb0FBIixudWxsLDIwXQ=="></a> is not clickable at point (273, 615). Other element would receive the click: <button class="e2moi " aria-label="Haut de page" jsaction="pane.wfvdle36;keydown:ripple.play;mousedown:ripple.play;ptrdown:ripple.play" jslog="133106; track:click;">...</button>
  (Session info: chrome=125.0.6422.78)
Stacktrace:
0   chromedriver            